In [ ]:
from PPO_train_test import PPO_train, PPO_train_udr, PPO_train_adr, PPO_test

TIMESTEPS = 1_500_000
SEED = 42

We want to deploy our agent in the real world that is represented by the Target environment, but we have only access to a simulator (source environment) which has a sistematic error wrt real word.

Since we want to close this sim2real gap we will experiment some techniques to address this problem.

To begin with, we will try a simple UDR. Then our focus will be implementing an ADR technique inspired to the OpenAI original paper where ADR is formally introduced in the RL world.

In the following experiments PPO, a policy-based RL algorithm, will be employed.

# Hopper Environment

Nota:
- In Target, l'hopper ha un torso più pesante. La massa elevata conferisce inerzia. L'inerzia agisce come un "filtro passa-basso" naturale: smorza le oscillazioni e rende il robot più stabile contro piccoli errori di controllo o rumore nelle azioni.

- In Source, l'hopper ha un torso molto più leggero. Con meno inerzia, il robot diventa "nervoso" (twitchy). Ogni piccola forza applicata dai motori si traduce in un'accelerazione angolare molto più rapida (F=m⋅a→a=F/m; se m scende, a sale). Questo rende molto più facile per il robot perdere l'equilibrio e cadere (terminando l'episodio prima dei 500 step).

I motori (attuatori) dell'Hopper hanno la stessa forza massima in entrambi gli ambienti, ma nel Source devono spingere meno massa.
Nel caso Source, il robot è "sovra-potenziato" rispetto al suo peso. Questo rende il paesaggio di ottimizzazione (la "superficie" che l'algoritmo PPO deve scalare) molto più frastagliato. Un'azione leggermente sbagliata nel Source porta a conseguenze catastrofiche (caduta) molto più velocemente che nel Target.
L'ambiente Hopper-v* standard di Gym (che corrisponde al tuo Target) è stato progettato e sintonizzato dai creatori di MuJoCo per avere proprietà fisiche che rendono la locomozione apprendibile e stabile. Modificando arbitrariamente una massa fondamentale come quella del torso (togliendo 1kg, che è una percentuale significativa del peso totale), si crea un robot "sbilanciato" o fisicamente meno adatto alla locomozione rispetto al design originale.

Per questo motivo, mentre target->target arriva sempre a 500 timesteps medi (con std=0) per episodio in test, source->source non ci riesce mai.

Training on SOURCE

In [ ]:
PPO_train(
    train_env_id='CustomHopper-source-v0',
    model_name=f'hopper_source_{SEED}',
    lr=3e-4,
    steps=TIMESTEPS,
    seed=SEED
)

SOURCE -> SOURCE

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-source-v0',
    model_name=f'hopper_source_{SEED}'
)

SOURCE -> TARGET

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=f'hopper_source_{SEED}'
)

Training on TARGET

In [ ]:
PPO_train(
    train_env_id='CustomHopper-target-v0',
    model_name=f'hopper_target_{SEED}',
    lr=3e-4,
    steps=TIMESTEPS,
    seed=SEED
)

TARGET -> TARGET

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=f'hopper_target_{SEED}'
)

## UDR

In [ ]:
model_name=f'hopper_source_udr_constant_50_medium_{SEED}'

In [ ]:
PPO_train_udr(
    train_env_id='CustomHopper-source-v0',
    model_name=model_name,
    lr=3e-4,
    lr_scheduler_type='constant',
    steps=TIMESTEPS,
    udr_range=0.5,
    net_size="medium", # [ small - medium - large ] -> [ 64 - 128 - 256 ]
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=model_name
)

## ADR (OpenAI Style)

In [ ]:
model_name = f'hopper_source_adr_constant_70_1300_40000_medium_{SEED}'

In [ ]:
PPO_train_adr(
    train_env_id='CustomHopper-source-v0',
    model_name=model_name,
    lr=3e-4,
    lr_scheduler_type="constant",
    steps=TIMESTEPS,
    starting_adr_range=0.05,
    objective_adr_range=0.5,
    increase_rate=0.05,
    reward_to_check=1300,
    check_frequency=40_000,
    net_size="medium",
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=model_name
)

## Alternative: SAC

In [ ]:
model_name = f'hopper_sac_source_adr_constant_50_1300_40000_medium_{SEED}'

In [ ]:
from SAC_train_test import *

SAC_train_adr(
    train_env_id='CustomHopper-source-v0',
    model_name=model_name,
    lr=3e-4,
    lr_scheduler_type="constant",
    steps=300_000,
    starting_adr_range=0.05,
    objective_adr_range=0.5,
    increase_rate=0.05,
    reward_to_check=1300,
    check_frequency=40_000,
    net_size="medium",
    seed=SEED
)

In [ ]:
mean, std = SAC_test(
    test_env_id='CustomHopper-target-v0',
    model_name=model_name
)

## Hopper Visualization

In [ ]:
from utils.visualize_agent import visualize

visualize(
    model_path=f"models/{model_name}",
    env_id="CustomHopper-target-v0"
)

# Walker2D Environment

Training on SOURCE

In [ ]:
PPO_train(
    train_env_id='CustomWalker-source-v0',
    model_name=f'walker_source_{SEED}',
    lr=3e-4,
    steps=TIMESTEPS,
    seed=SEED
)

SOURCE -> SOURCE

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomWalker-source-v0',
    model_name=f'walker_source_{SEED}'
)

SOURCE -> TARGET

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomWalker-target-v0',
    model_name=f'walker_source_{SEED}'
)

Training on TARGET

In [ ]:
PPO_train(
    train_env_id='CustomWalker-target-v0',
    model_name=f'walker_target_{SEED}',
    lr=3e-4,
    steps=TIMESTEPS,
    seed=SEED
)

TARGET -> TARGET

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomWalker-target-v0',
    model_name=f'walker_target_{SEED}'
)

## UDR

In [ ]:
model_name=f'walker_source_udr_constant_50_medium_{SEED}'

In [ ]:
PPO_train_udr(
    train_env_id='CustomWalker-source-v0',
    model_name=model_name,
    lr=3e-4,
    lr_scheduler_type='constant',
    steps=TIMESTEPS,
    udr_range=0.5,
    net_size="medium", # [ small - medium - large ] -> [ 64 - 128 - 256 ]
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomWalker-target-v0',
    model_name=model_name
)

## ADR (OpenAI Style)

In [ ]:
model_name = f'ppo_source_adr_constant_50_1300_40000_medium_{SEED}'

In [ ]:
PPO_train_adr(
    train_env_id='CustomWalker-source-v0',
    model_name=model_name,
    lr=3e-4,
    lr_scheduler_type="constant",
    steps=TIMESTEPS,
    starting_adr_range=0.05,
    objective_adr_range=0.5,
    increase_rate=0.05,
    reward_to_check=1300,
    check_frequency=40_000,
    net_size="medium",
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomWalker-target-v0',
    model_name=model_name
)

## Walker Visualization

In [ ]:
from utils.visualize_agent import visualize

model_name = f"walker_source_{SEED}"

visualize(
    model_path=f"models/{model_name}",
    env_id="CustomWalker-target-v0"
)